# SpaceX Falcon 9 First Stage Landing Prediction: Map

## Objectives
- Mark all launch sites on a map
- Mark the success/failed launches for each site on the map
- Calculate the distances between a launch site to its proximities

### Import libraries

In [8]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [2]:
# Display all columns of a dataframe
pd.set_option("display.max_columns", None)
# Display all of the data in a feature
pd.set_option("display.max_colwidth", None)

import warnings
warnings.filterwarnings("ignore")

### Import dataset with latitude and longitude

In [3]:
# Import the dataset
df = pd.read_csv("spacex_dataset_cleaned.csv")

In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = df[['LaunchSite', 'Latitude', 'Longitude', 'Class']]
launch_sites_df = spacex_df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df = launch_sites_df[['LaunchSite', 'Latitude', 'Longitude']]
launch_sites_df

,LaunchSite,Latitude,Longitude
0,CCSFS SLC 40,28.561857,-80.577366
1,KSC LC 39A,28.608058,-80.603956
2,VAFB SLC 4E,34.632093,-120.610829


### Visualize the Launch Site on the map

In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [19]:
# Initialize map
center_coords = [launch_sites_df[['Latitude']].max() * 0.5 + launch_sites_df[['Latitude']].min() * 0.5, launch_sites_df[['Longitude']].max() * 0.5 + launch_sites_df[['Longitude']].min() * 0.5]
site_map = folium.Map(location=center_coords, zoom_start=4)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for _, launch_site in launch_sites_df.iterrows():
    coord = [launch_site['Latitude'], launch_site['Longitude']]
    ls_name = launch_site['LaunchSite']
    # Create a blue circle at site's coordinate with a popup label showing its name
    circle = folium.Circle(coord, radius=1000, color='#FF0000', fill=True).add_child(folium.Popup(ls_name))
    # Create a blue circle at site's coordinate with a icon showing its name
    marker = folium.map.Marker(
        coord,
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(30,30),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#FF0000;"><b>%s</b></div>' % ls_name,
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map  # display map

### Visualize success/failed launches for each site on the map

In [20]:
spacex_df.tail(10)

,LaunchSite,Latitude,Longitude,Class
80,CCSFS SLC 40,28.561857,-80.577366,1
81,CCSFS SLC 40,28.561857,-80.577366,1
82,CCSFS SLC 40,28.561857,-80.577366,1
83,CCSFS SLC 40,28.561857,-80.577366,1
84,CCSFS SLC 40,28.561857,-80.577366,1
85,KSC LC 39A,28.608058,-80.603956,1
86,KSC LC 39A,28.608058,-80.603956,1
87,KSC LC 39A,28.608058,-80.603956,1
88,CCSFS SLC 40,28.561857,-80.577366,1
89,CCSFS SLC 40,28.561857,-80.577366,1


In [30]:
# Create marker object
marker_cluster = MarkerCluster().add_to(site_map)

In [31]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

In [32]:
spacex_df['MarkerColor'] = spacex_df['Class'].apply(assign_marker_color)
spacex_df.tail(10)

,LaunchSite,Latitude,Longitude,Class,MarkerColor
80,CCSFS SLC 40,28.561857,-80.577366,1,green
81,CCSFS SLC 40,28.561857,-80.577366,1,green
82,CCSFS SLC 40,28.561857,-80.577366,1,green
83,CCSFS SLC 40,28.561857,-80.577366,1,green
84,CCSFS SLC 40,28.561857,-80.577366,1,green
85,KSC LC 39A,28.608058,-80.603956,1,green
86,KSC LC 39A,28.608058,-80.603956,1,green
87,KSC LC 39A,28.608058,-80.603956,1,green
88,CCSFS SLC 40,28.561857,-80.577366,1,green
89,CCSFS SLC 40,28.561857,-80.577366,1,green


In [33]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# Create a Marker object with its coordinate and customize the Marker's icon property to indicate if this launch was successed or failed,
for index, record in spacex_df.iterrows():
    folium.Marker(
        location=[record['Latitude'], record['Longitude']],
        icon=folium.Icon(color=record['MarkerColor'], icon_color=record['MarkerColor'],
        popup=record['Class'])
    ).add_to(marker_cluster)

site_map

### Calculate the distances between a launch site to its proximities

In [34]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [35]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [36]:
# find coordinate of the closet coastline
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

launch_site_lat, launch_site_lon = 28.563197, -80.576820
coastline_lat, coastline_lon = 28.56319, -80.56785

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

0.8762983388668405

In [37]:
# Display the distance between coastline point and launch site using the icon property 
distance_marker = folium.Marker(
   location=[coastline_lat, coastline_lon],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   )

In [38]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]], weight=1)
site_map.add_child(lines)

In [39]:
city_lat, city_lon = 28.53, -81.38  # coords for Orlando

line2=folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]], weight=3)
site_map.add_child(line2)